In [26]:
import os
import sys
#dir_path = os.path.dirname(os.path.realpath(__file__))
dir_path = "/Users/neda/HiCPlus_pytorch/src"
model_path = dir_path + "/models"
utils_path = dir_path + "/utils"
sys.path.insert(0, model_path)
sys.path.insert(0, utils_path)
import model2
import utils

import numpy as np
import matplotlib.pyplot as plt
import pickle
import gzip
from torch.utils import data
import torch
import torch.optim as optim
from torch.autograd import Variable
from time import gmtime, strftime
import torch.nn as nn
from scipy.stats.stats import pearsonr
import argparse
import cooler


args = {"low_res_cool_path": "/Users/neda/prostate-samples/PCa13266.multi-res.cool"
       , "high_res_cool_path": "/Users/neda/prostate-samples/HighRes-PCa13266.multi-res.cool"
       , "output_path": "/Users/neda/prostate-samples/PCa13266-down/"
       , "model_path": "/Users/neda/HiCPlus_pytorch/src/learned-networks"
       , "resolution": "10000"
       , "model_name": "chr1-17-PCa13266(up-wo-min)"
       , "chrN_min": "1"
       , "chrN_max": "17"
       , "frames_data_path": "/Users/neda/prostate-samples/divided-data"
       , "sample_name": "PCa13266"}

high_res_cool = cooler.Cooler(args['high_res_cool_path'] + '::/resolutions/' + str(args['resolution']))
p = high_res_cool.pixels()
total_num_reads = sum(p[:,].iloc[:,2])
low_res_cool = cooler.Cooler(args['low_res_cool_path'] + '::/resolutions/' + str(args['resolution']))
p2 = low_res_cool.pixels()
low_total_num_reads = sum(p2[:,].iloc[:,2])

# Obs1: when we fetch a specific chromosome it means first columns belong to regions in that chromosome but
# second column regions are through whole genome
# Obs2: reads are not considered twice in files, for example when we fetch chr2 pixels, there are not interactions
# between chr2 and chr1 any more.
# Obs3: number of intra reads: 125015861, whole reads: 153752070 (in low resolution sample)

vec_of_prob = []
for chrName in high_res_cool.chromnames:
    vec_of_prob.extend(high_res_cool.matrix(balance = False, as_pixels = True).fetch(chrName).iloc[:,2])
num_inter_reads = total_num_reads - sum(vec_of_prob)
vec_of_prob.append(num_inter_reads)
vec_of_prob = [p/total_num_reads for p in vec_of_prob]
down_sampled_counts = np.random.multinomial(low_total_num_reads, vec_of_prob)

In [28]:

down_sampled_counts[0:10,]

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])

In [35]:
vec_of_prob[0:10]

[1.3615334263679109e-09,
 1.3615334263679109e-09,
 1.3615334263679109e-09,
 2.7230668527358217e-09,
 1.3615334263679109e-09,
 1.3615334263679109e-09,
 1.3615334263679109e-09,
 1.3615334263679109e-09,
 1.3615334263679109e-09,
 4.0846002791037326e-09]

In [5]:
import cooler
#high_res_cool = cooler.Cooler("/Users/neda/prostate-samples/HighRes-PCa13266.multi-res.cool::/resolutions/10000")
import numpy as nphigh_res_cool.matrix(balance = False, as_pixels = True).fetch("chr1").iloc[1000:1100]

SyntaxError: invalid syntax (<ipython-input-5-6d44ab1fb44a>, line 3)

In [7]:

import numpy as np
a = np.zeros([2,3])
a[0,1] = 2

In [11]:
import cooler
c = cooler.Cooler('/Users/neda/prostate-samples/PCa13266.multi-res.cool::/resolutions/10000')
c.pixels()[:,].loc[1398]

bin1_id        75
bin2_id    187799
count           1
Name: 1398, dtype: int64

In [12]:
for (i,j) in zip([1,2,3],[4,5,6,7]):
    print(i,j)

1 4
2 5
3 6


In [5]:
import numpy as np
r = np.load("/Users/neda/HiCPlus_pytorch/data/divided-data/GM12878_primary/10kb_resolution/chr1-17-down16.npy")
r2 = np.load("/Users/neda/HiCPlus_pytorch/data/divided-data/GM12878_primary/10kb_resolution/chr1-17.npy")
i = np.load("/Users/neda/HiCPlus_pytorch/data/divided-data/GM12878_primary/10kb_resolution/chr1-17-down16-index.npy")
i2 = np.load("/Users/neda/HiCPlus_pytorch/data/divided-data/GM12878_primary/10kb_resolution/chr1-17-index.npy")

In [6]:
i2.shape

(92151, 3)

In [7]:
print(r[1,])
print(r2[10000,])
print(i[10000,])
print(i2[10000,])

[[0 0 1 ... 0 1 0]
 [0 0 1 ... 0 0 0]
 [0 1 2 ... 0 0 0]
 ...
 [2 0 3 ... 0 1 0]
 [0 0 1 ... 3 0 1]
 [0 1 3 ... 2 1 0]]
[[29 12 20 ... 14 10  9]
 [34 18 32 ... 19 23  7]
 [22 23 27 ... 14 17 19]
 ...
 [41 36 59 ... 32 43 40]
 [84 57 90 ... 46 56 59]
 [95 77 89 ... 64 79 62]]
[    4 17175 17200]
[   3 1400 1425]


In [12]:
i[20000,]

array([  8,  75, 125])

In [13]:
i2[20000,]

array([  10, 9500, 9550])

In [42]:
r = np.load("/Users/neda/HiCPlus_pytorch/data/divided-data/GM12878-down5-chr1-17.npy")
r2 = np.load("/Users/neda/HiCPlus_pytorch/data/divided-data/GM12878-chr1-17.npy")

In [43]:
print(r[60000,])
print(r2[60000,])

[[3 4 2 ... 0 0 0]
 [0 1 1 ... 1 0 0]
 [2 2 2 ... 0 1 0]
 ...
 [0 1 0 ... 0 1 0]
 [2 0 2 ... 3 0 0]
 [1 0 1 ... 1 1 0]]
[[ 9  6  4 ...  3  3  0]
 [ 4  4  6 ...  2  0  2]
 [ 7  8  5 ...  2  4  1]
 ...
 [ 7  4 12 ...  4  1  1]
 [ 9  1  9 ...  6  4  1]
 [ 7  6 11 ...  1  1  0]]


In [45]:
i = np.load("/Users/neda/HiCPlus_pytorch/data/divided-data/GM12878-down5-chr1-17-index.npy")
i2 = np.load("/Users/neda/HiCPlus_pytorch/data/divided-data/GM12878-chr1-17-index.npy")

In [46]:
print(i[70000,])
print(i2[70000,])

[ 12 725 900]
[ 12 725 900]


In [2]:
import numpy as np
m = np.load("/Users/neda/temp.npy")

In [3]:
m[1000,]

array([[103, 119,  73, ...,   9,  18,  17],
       [ 89, 100,  65, ...,  16,   8,  17],
       [ 98,  93,  66, ...,   8,  18,  20],
       ...,
       [153, 168, 153, ...,  69,  67,  70],
       [102,  99,  84, ...,  41,  35,  51],
       [120, 118, 105, ...,  61,  55,  57]], dtype=int16)